In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm import tqdm
a=0
ids = []
for n in tqdm(open('/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta')):
    if n.startswith('>'):
        a+=1
        ids.append(n.split(' ')[0].replace('>',''))
print(a)
open('ids_test.txt','w').write('\n'.join(ids))

In [ ]:
def create_id_go_file_optimized(df_filtered, ids, output_file='id_go_pairs.txt', verbose=True):
    """
    Optimized version using Polars operations for better performance with large datasets.
    
    Args:
        df_filtered: Polars dataframe with 'db_object_id' and 'go_id' columns
        ids: List of UniProt IDs to extract
        output_file: Output file path
        verbose: Whether to print warnings for missing IDs
    
    Returns:
        Tuple of (found_ids, missing_ids)
    """
    
    print(f"Filtering dataframe for {len(ids)} IDs...")
    
    # Filter dataframe for all IDs at once (much faster than individual filters)
    filtered_subset = df_filtered.filter(pl.col('db_object_id').is_in(ids))
    
    # Get unique IDs that were actually found
    found_ids = filtered_subset['db_object_id'].unique().to_list()
    
    # Find missing IDs
    missing_ids = list(set(ids) - set(found_ids))
    
    # Sort the subset by ID for organized output
    filtered_subset = filtered_subset.sort('db_object_id')
    
    # Write to file with progress bar
    with open(output_file, 'w') as f:
        # Group by ID for progress tracking
        grouped = filtered_subset.group_by('db_object_id', maintain_order=True)
        
        for (id_value,), group_df in tqdm(grouped, desc="Writing ID-GO pairs", total=len(found_ids)):
            for row in group_df.iter_rows():
                go_id_idx = group_df.columns.index('go_id')
                go_term = row[go_id_idx]
                f.write(f"{id_value}\t{go_term}\t1\n")
    
    # Print summary
    print(f"\n✅ Completed processing {len(ids)} IDs")
    print(f"   - Found: {len(found_ids)} IDs")
    print(f"   - Missing: {len(missing_ids)} IDs")
    
    if missing_ids and verbose:
        print(f"\n⚠️  Missing IDs ({len(missing_ids)}):")
        for missing_id in sorted(missing_ids)[:10]:  # Show first 10
            print(f"   - {missing_id}")
        if len(missing_ids) > 10:
            print(f"   ... and {len(missing_ids) - 10} more")
    
    # Save missing IDs to file for reference
    if missing_ids:
        missing_file = output_file.replace('.tsv', '_missing_ids.txt')
        with open(missing_file, 'w') as f:
            for missing_id in sorted(missing_ids):
                f.write(f"{missing_id}\n")
        print(f"\n💾 Missing IDs saved to: {missing_file}")
    
    return found_ids, missing_ids


In [ ]:
import polars as pl
df_filtered = pl.read_csv('/kaggle/input/goa-uniprot-all/selection_ids_goa_uniprot_all.csv')
df_filtered.head()

In [ ]:
[n for n in df_filtered['qualifier'].unique() if 'NOT' in n]

In [ ]:
print(df_filtered.shape)
df_filtered = df_filtered.filter(~pl.col('qualifier').str.contains('NOT'))
print(df_filtered.shape)

In [ ]:
found, missing = create_id_go_file_optimized(
        df_filtered, 
        ids, 
        output_file='submission_goa.tsv',
        verbose=True
    )
    
print("\n✨ Done! Check the output files.")

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
kaggle_key = user_secrets.get_secret("kaggle_key")

In [ ]:
import os
import json

# Set your Kaggle credentials as environment variables
# You need your username and API key from kaggle.json
os.environ['KAGGLE_USERNAME'] = 'mtinti'
os.environ['KAGGLE_KEY'] = kaggle_key

In [ ]:
!kaggle kernels output sasaleaf/merge-of-2submission-lb-0-25 -p /kaggle/working/merge

In [ ]:
!cat /kaggle/working/merge/submission.tsv 'submission_goa.tsv' > 'submission.tsv'

In [ ]:
!head 'submission.tsv'

In [ ]:
! rm -r /kaggle/working/merge
! rm 'submission_goa.tsv'
! rm 'submission_goa_missing_ids.txt'